In [ ]:
from pprint import pprint
from paddlenlp import Taskflow

In [ ]:
text = '感染者0: 现住：顺义区李桥镇后桥村 主要活动轨迹：4月6日-9日，每天8:00左右从居住地骑电动车到顺义区裕龙六区33号楼改水电，18:00左右返回居住地。4月10日在家休息。4月11日-21日，每天6:00左右从居住地骑电动车到东城区南门仓胡同小区改水电，18:00左右骑电动车返回居住地。其间分别于17日和19日11:00左右，在装修现场与22日通报的确诊病例有接触。4月22日，7:00左右从居住地骑电动车出发到东城区南门仓胡同小区取工具，9:30左右到东城区和平里三区业主家后再次返回南门仓胡同小区，14:00左右骑电动车离开返回居住地，20:00左右作为密接人员被转运到集中隔离医学观察点。'

In [ ]:
schema = ['感染者','日期', '时间', '地点']

In [ ]:
ie = Taskflow('information_extraction', schema=schema)
pprint(ie([text]))

In [ ]:
my_ie = Taskflow("information_extraction", schema=schema, task_path='./checkpoint/model_best')
pprint(my_ie(text))
